In [ ]:
import tensorflow as tf
from tensorflow.keras import datasets, layers, models, regularizers, optimizers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np

In [ ]:
EPOCHS=50
NUM_CLASSES = 10

In [ ]:
def load_data():
    (x_train, y_train), (x_test, y_test) = datasets.cifar10.load_data()
    x_train = x_train.astype('float32')
    x_test = x_test.astype('float32')
 
    # normalize 
    # 데이타 표준화 시킴
    mean = np.mean(x_train,axis=(0,1,2,3))
    std = np.std(x_train,axis=(0,1,2,3))
    print(f"mean : {mean}, std : {std}")

    x_train = (x_train-mean)/(std+1e-7)
    x_test = (x_test-mean)/(std+1e-7)
 
    y_train =  tf.keras.utils.to_categorical(y_train,NUM_CLASSES)
    y_test =  tf.keras.utils.to_categorical(y_test,NUM_CLASSES)

    return x_train, y_train, x_test, y_test

In [ ]:
def build_model(): 
    model = models.Sequential()
    
    #1st blocl
    model.add(layers.Conv2D(32, (3,3), padding='same', 
        input_shape=x_train.shape[1:], activation='relu'))
    model.add(layers.BatchNormalization())
    model.add(layers.Conv2D(32, (3,3), padding='same', activation='relu'))
    model.add(layers.BatchNormalization())
    model.add(layers.MaxPooling2D(pool_size=(2,2)))
    model.add(layers.Dropout(0.2)) #20%

    #2nd block
    model.add(layers.Conv2D(64, (3,3), padding='same', activation='relu'))
    model.add(layers.BatchNormalization())
    model.add(layers.Conv2D(64, (3,3), padding='same', activation='relu'))
    model.add(layers.BatchNormalization())
    model.add(layers.MaxPooling2D(pool_size=(2,2)))
    model.add(layers.Dropout(0.3)) #30%

    #3d block 
    model.add(layers.Conv2D(128, (3,3), padding='same', activation='relu'))
    model.add(layers.BatchNormalization())
    model.add(layers.Conv2D(128, (3,3), padding='same', activation='relu'))
    model.add(layers.BatchNormalization())
    model.add(layers.MaxPooling2D(pool_size=(2,2)))
    model.add(layers.Dropout(0.4)) #40%

    #dense  
    model.add(layers.Flatten())
    model.add(layers.Dense(NUM_CLASSES, activation='softmax'))
    return model

    model.summary()

In [ ]:
(x_train, y_train, x_test, y_test) = load_data()
model = build_model()

170500096/170498071 [==============================] - 4s 0us/step
mean : 120.70748138427734, std : 64.1500244140625


In [ ]:
model.summary() #parmas는 이전 모델이 더 많은데, 이전 모델은 중간에 한 번 막았고 여기서는 플랫한 뒤 바로 출력해버려서 그럼

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 32, 32, 32)        896       
_________________________________________________________________
batch_normalization (BatchNo (None, 32, 32, 32)        128       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 32, 32, 32)        9248      
_________________________________________________________________
batch_normalization_1 (Batch (None, 32, 32, 32)        128       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 16, 16, 32)        0         
_________________________________________________________________
dropout (Dropout)            (None, 16, 16, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 16, 16, 64)        1

In [ ]:
model.compile(loss='categorical_crossentropy', 
              optimizer='RMSprop', # sgd, adam ...등등
              metrics=['accuracy'])

In [ ]:
import keras
early_stopping_cb = keras.callbacks.EarlyStopping(patience=3, restore_best_weights=True)

In [ ]:
#train
batch_size = 64
model.fit(x_train, y_train, batch_size=batch_size,
    epochs=EPOCHS, validation_data=(x_test,y_test),
    callbacks=[early_stopping_cb]) #epochs : 하나가 다 끝났을 때, batch_size : batch_size만큼 순환을 하고 나면 1epoch

Epoch 1/50
782/782 [==============================] - 41s 11ms/step - loss: 1.7746 - accuracy: 0.4682 - val_loss: 1.3428 - val_accuracy: 0.5648
Epoch 2/50
782/782 [==============================] - 8s 10ms/step - loss: 1.1155 - accuracy: 0.6451 - val_loss: 0.9299 - val_accuracy: 0.6742
Epoch 3/50
782/782 [==============================] - 8s 10ms/step - loss: 0.8769 - accuracy: 0.7098 - val_loss: 0.8511 - val_accuracy: 0.7312
Epoch 4/50
782/782 [==============================] - 8s 10ms/step - loss: 0.7555 - accuracy: 0.7465 - val_loss: 0.7134 - val_accuracy: 0.7574
Epoch 5/50
782/782 [==============================] - 8s 11ms/step - loss: 0.6645 - accuracy: 0.7729 - val_loss: 0.6684 - val_accuracy: 0.7721
Epoch 6/50
782/782 [==============================] - 8s 11ms/step - loss: 0.5987 - accuracy: 0.7944 - val_loss: 0.6249 - val_accuracy: 0.7898
Epoch 7/50
782/782 [==============================] - 8s 11ms/step - loss: 0.5420 - accuracy: 0.8138 - val_loss: 0.6277 - val_accuracy: 0.794

In [ ]:
score = model.evaluate(x_test, y_test,
                     batch_size=batch_size)
print("\nTest score:", score[0])
print('Test accuracy:', score[1])

157/157 [==============================] - 1s 5ms/step - loss: 0.4721 - accuracy: 0.8434

Test score: 0.47214293479919434
Test accuracy: 0.8434000015258789
